# Apcliações do SVM

## Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

## Importando o dataset

In [ ]:
from sklearn.datasets import load_wine

In [ ]:
wine = load_wine()

### Leitura dos dados

In [ ]:
X = wine.data

In [ ]:
y = wine.target

In [ ]:
X.shape, y.shape

In [ ]:
wine.feature_names

In [ ]:
wine.target_names

### Dividindo os dados

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
ts = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts, random_state=42)

## Treinando o SVM

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
model_clean = make_pipeline(StandardScaler(), SVC())

## SVM

### Procurando os `Kernels`

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'svc__C': [0.01, 0.5, 1, 5],
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'svc__decision_function_shape':  ['ovo', 'ovr']
}
grid = GridSearchCV(model_clean, param_grid)

%time grid.fit(X_train, y_train)
print(grid.best_params_)

In [ ]:
model_clean = grid.best_estimator_
y_fit = model_clean.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_fit,
                            target_names=wine.target_names))

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, y_fit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=True,
            xticklabels=wine.target_names,
            yticklabels=wine.target_names,cmap=plt.cm.Blues)
plt.xlabel('Classes Reais')
plt.ylabel('Classes Preditos');

### Testando os `kernels` e parâmetros

In [ ]:
kernel_list = ['linear', 'poly', 'rbf', 'sigmoid']

In [ ]:
def gs_best_params(C_list, gamma_list, decision_list, X_test):
    
    for kernel in kernel_list:
        model = make_pipeline(StandardScaler(), SVC(kernel=kernel))
        
        if kernel == 'linear':
            param_grid = {
                'svc__C': C_list,
                'svc__decision_function_shape':  decision_list
            }
        else:
            param_grid = {
                'svc__C': C_list,
                'svc__gamma': gamma_list,
                'svc__decision_function_shape':  decision_list
            }
        
        grid = GridSearchCV(model, param_grid)
        print(f"### Grid Search ({model.named_steps.svc.kernel}) ###")
        %time grid.fit(X_train, y_train)
        print(grid.best_params_)
        model = grid.best_estimator_
        y_fit = model.predict(X_test)
        print("### Classification report ###")
        print(classification_report(
            y_test, y_fit, target_names=wine.target_names)
        )
        mat = confusion_matrix(y_test, y_fit)
        sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=True,
                    xticklabels=wine.target_names,
                    yticklabels=wine.target_names,cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.xlabel('Classes Reais')
        plt.ylabel('Classes Preditos');
        plt.show()
        print("\n")

In [ ]:
C_list = [0.001, 0.05, 1, 5, 10, 30, 40]
gamma_list = [0.001, 0.005, 0.01, 0.05, 1]
decision_list = ['ovo', 'ovr']

gs_best_params(C_list, gamma_list, decision_list, X_test)